In [1]:
from qiskit import QuantumCircuit, Aer, assemble
from math import pi
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import matplotlib.pyplot as plt

In [2]:
# set initial state
n=2
d=2
qc = QuantumCircuit(2)

In [3]:
theta_rzz = 0 ##
theta = np.random.uniform(0, 2*pi, (3,2,3))
print(theta)

[[[0.25928348 5.36953415 5.89058306]
  [4.72587927 0.82669428 0.92253431]]

 [[1.45450555 5.91739808 3.97173378]
  [2.93307662 0.67471872 2.76567737]]

 [[3.38699796 0.78028535 0.5918956 ]
  [4.76076725 2.46259796 1.55402114]]]


In [5]:
def single_qubit_gate(qc, rtheta, qubit):
    qc.rx(rtheta[0], qubit)
    qc.ry(rtheta[1], qubit)
    qc.rx(rtheta[2], qubit)
    return qc

In [10]:
depth=2
def parametrized_circuit(theta, theta_rzz):
    qc = QuantumCircuit(2)
    for i in range(depth+1):
        for j in range(n):
            single_qubit_gate(qc, theta[i, j], j)
            print("single")
        if i==depth:
            break
        else:
            qc.rzz(theta_rzz, 0, 1)
            print("source")
            
    
parametrized_circuit(theta, theta_rzz)

single
single
source
single
single
source
single
single


In [35]:
dimension = 4
n=2
depth=2

def rx(angle):
    return np.array([[np.cos(angle/2), -1j*np.sin(angle/2)], [-1j*np.sin(angle/2), np.cos(angle/2)]])

def ry(angle):
    return np.array([[np.cos(angle/2), -1*np.sin(angle/2)], [np.sin(angle/2), np.cos(angle/2)]])


def single_gate(alpha, beta, gamma):
    return rx(alpha)@ry(beta)@rx(gamma)
    
def U(theta, source_theta):
    theta = theta.detach().numpy()
    uop = np.identity(dimension)
    for i in range(depth+1):
        single=[]
        for j in range(n):
            alpha, beta, gamma = theta[i, j]
            single.append(single_gate(alpha, beta, gamma))
            print("single")
            print(single)
        if i==depth:
            uop = uop @ np.tensordot(single[1], single[0], axes=0)
        else:
            print(uop.shape)
            print(single[0])
            print(single[1])
            print(np.tensordot(single[1], single[0], 0).shape)
            print(np.tensordot(single[1], single[0], 0))
            uop = uop @ np.tensordot(single[1], single[0], axes=0) @ np.diag( [np.exp(-1j*source_theta/2), np.exp(1j*source_theta/2), np.exp(1j*source_theta/2), np.exp(-1j*source_theta/2) ])
            print("source")
    theta = Variable(torch.FloatTensor(theta), requires_grad=True)
    return uop, theta

target = np.array([[1,0, 0 ,0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
print(target)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]


In [32]:
def fidelity(theta, source_theta, dimension):
    uop, theta = U(theta, source_theta)
    fid = 1/(dimension+1)*( np.trace(target @ uop)/dimension + 1)
    return fid, theta

initial_theta = np.random.uniform(0, 2*pi, (3,2,3))

In [33]:
import torch
from torch.autograd import Variable

In [36]:
epochs = 10
lr = 0.1

theta = Variable(torch.FloatTensor(initial_theta), requires_grad=True)
for e in range(epochs):
    fid, theta = fidelity(theta, np.pi/4, 4)
    loss = 1 - fid
    loss.backward()
    print(loss)

single
[array([[ 0.36941388-0.70672481j, -0.17974801+0.57598964j],
       [ 0.17974801+0.57598964j,  0.36941388+0.70672481j]])]
single
[array([[ 0.36941388-0.70672481j, -0.17974801+0.57598964j],
       [ 0.17974801+0.57598964j,  0.36941388+0.70672481j]]), array([[ 0.82638053+0.12564839j, -0.50174012-0.22263097j],
       [ 0.50174012-0.22263097j,  0.82638053-0.12564839j]])]
(4, 4)
[[ 0.36941388-0.70672481j -0.17974801+0.57598964j]
 [ 0.17974801+0.57598964j  0.36941388+0.70672481j]]
[[ 0.82638053+0.12564839j -0.50174012-0.22263097j]
 [ 0.50174012-0.22263097j  0.82638053-0.12564839j]]
(2, 2, 2, 2)
[[[[ 0.39407528-0.53760737j -0.22091243+0.45340158j]
   [ 0.07616809+0.49857168j  0.21647761+0.63043989j]]

  [[-0.3426886 +0.27234922j  0.21841993-0.24897964j]
   [ 0.03804635-0.32901459j -0.02801093-0.43683516j]]]


 [[[ 0.02801093-0.43683516j  0.03804635+0.32901459j]
   [ 0.21841993+0.24897964j  0.3426886 +0.27234922j]]

  [[ 0.21647761-0.63043989j -0.07616809+0.49857168j]
   [ 0.22091243+0.4

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)